In [1]:
# 假设一个小时后要考默写这个代码
# 有时候背下来真的是很好的办法，先背下来慢慢理解

import torch
from torch import nn #neural network
from torch.utils.data import DataLoader
#每个dataset都可以被相同的dataloader类加载
from torchvision import datasets
#dataset包含len和dataset[i]，即数据集有多大和数据集第i个样本是什么
#dataloader负责batch_size, shuffle等数据加载的方式
from torchvision.transforms import ToTensor

In [2]:
# 导入完包，接下来创建对象实例
training_data = datasets.FashionMNIST( #注意这里括号里全有逗号
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [3]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
# 创建完数据集后创建数据加载器
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [6]:
# 一次小试验
for X, y in test_dataloader:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape}")
    break

Shape of X: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


In [7]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [9]:
# 定义模型
# 模型为什么一定是类？
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
# 创建一个模型实例
model = NeuralNetwork().to(device) #不to就还在cpu上
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
  )
)


In [12]:
# 创建loss function 和 optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
# 1e-3 = 1*10^-3 = 0.001
# learning rate学习率
# 这里只是拿走了model的参数引用

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        #不是枚举，内置函数，同时拿到索引和元素
        X, y = X.to(device), y.to(device)
        pred = model(X) #计算预测值
        loss = loss_fn(pred, y) #计算loss
        loss.backward() #反向传播
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0: #每处理100次batch打印一次当前训练进度和loss
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:7f} [{current:>5}/{size:>5}]")


In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Test Error: 
 Accuracy: 64.9%, Avg loss: 1.061061 

Done!


In [17]:
# saving models
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [18]:
# loading models
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
